In [ ]:
# !pip install -q mediapipe==0.10.0
!pip install --upgrade -q gspread

from google.colab.patches import cv2_imshow
from google.colab import auth
import gspread
import pandas as pd
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor
from skimage import io, transform
from tqdm import tqdm
auth.authenticate_user()
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.4 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
print(gc.open('hands and palms loras'))

<Spreadsheet 'hands and palms loras' id:1GHwH5am7o7RKga8FPruSKLgMTXIWG_wEMKk__L0J72A>


In [ ]:
worksheet = gc.open('hands and palms loras').worksheet('scale1').get("A2:I625")
df = pd.DataFrame.from_records(worksheet)
df = df.rename({0:'rank', 1:'checkpoint', 2:'side',3:'gender', 4:'hand', 5:'image', 6: 'bad', 7:'ok', 8:'good'}, axis=1)
df



,rank,checkpoint,side,gender,hand,image,bad,ok,good
0,64,30000,front,male,right,5,1,None,None
1,64,30000,front,male,right,4,1,None,None
2,64,30000,front,male,right,3,1,None,None
3,64,30000,front,male,right,2,,1,None
4,64,30000,front,male,right,1,1,None,None
...,...,...,...,...,...,...,...,...,...
619,None,0,back,female,left,4,1,None,None
620,None,0,back,female,left,3,,1,None
621,None,0,back,female,left,2,1,None,None
622,None,0,back,female,left,1,1,None,None


In [ ]:
!pip install diffusers torchmetrics
from tqdm import tqdm
from PIL import Image
import tensorflow as tf
import numpy as np
import torchvision
import torch

from torchmetrics.functional.multimodal import clip_score
from functools import partial

clip_score_fn = partial(clip_score, model_name_or_path="openai/clip-vit-base-patch16")

def calculate_clip_score(image, prompts):
    # images_int = (images * 255).astype("uint8")
    # clip_score = clip_score_fn(torch.from_numpy(images_int).permute(0, 3, 1, 2), prompts).detach()
    clip_score = clip_score_fn(torchvision.transforms.functional.pil_to_tensor(image).unsqueeze(0), prompts).detach()
    return float(clip_score)


for index, row in tqdm([(index, row) for (index, row) in df.iterrows()]):
  if row['rank'] != 'None':
    path = f'/content/drive/MyDrive/LoRA/inference/hands and palms r{row["rank"]} lora/{row["side"]}_of_fair-skinned_{row["gender"]}_{row["hand"]}_hand_chkpt{row["checkpoint"]}_steps30_{row["image"]}.png'
  else:
    path = f'/content/drive/MyDrive/LoRA/inference/hands and palms r{row["rank"]} lora/{row["side"]}_of_fair-skinned_{row["gender"]}_{row["hand"]}_hand_chkpt5000_steps30_{row["image"]}.png'
  image = Image.open(path)
  df.loc[index, 'clip'] = calculate_clip_score(image, [f'{row["side"]} of fair-skinned {row["gender"]} {row["hand"]} hand'])


  0%|          | 0/624 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/4.10k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/599M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/961k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

100%|██████████| 624/624 [37:35<00:00,  3.61s/it]


In [ ]:
df

,rank,checkpoint,side,gender,hand,image,bad,ok,good,clip
0,64,30000,front,male,right,5,1,None,None,29.845884
1,64,30000,front,male,right,4,1,None,None,30.810965
2,64,30000,front,male,right,3,1,None,None,30.695475
3,64,30000,front,male,right,2,,1,None,28.869411
4,64,30000,front,male,right,1,1,None,None,29.935408
...,...,...,...,...,...,...,...,...,...,...
619,None,0,back,female,left,4,1,None,None,29.004501
620,None,0,back,female,left,3,,1,None,27.531591
621,None,0,back,female,left,2,1,None,None,26.427406
622,None,0,back,female,left,1,1,None,None,26.062202


In [ ]:
gc.open('hands and palms loras').worksheet('scale1').update('M2:M625', df[['clip']].values.tolist())

<ipython-input-11-1368ffff22ec>:1: DeprecationWarning: [Deprecated][in version 6.0.0]: Method signature's arguments 'range_name' and 'values' will change their order. We recommend using named arguments for minimal impact. In addition, the argument 'values' will be mandatory of type: 'List[List]'. (ex) Worksheet.update(values = [[]], range_name=) 
  gc.open('hands and palms loras').worksheet('scale1').update('M2:M625', df[['clip']].values.tolist())


{'spreadsheetId': '1GHwH5am7o7RKga8FPruSKLgMTXIWG_wEMKk__L0J72A',
 'updatedRange': 'scale1!M2:M625',
 'updatedRows': 624,
 'updatedColumns': 1,
 'updatedCells': 624}

In [ ]:
df

,rank,checkpoint,side,gender,hand,image,bad,ok,good,mediapipe_highconf,weighted human,mediapipe
0,64,30000,front,male,right,5,1,None,None,0.000000,0.0,0.792618
1,64,30000,front,male,right,4,1,None,None,0.000000,0.0,0.700885
2,64,30000,front,male,right,3,1,None,None,0.000000,0.0,0.972215
3,64,30000,front,male,right,2,,1,None,0.000000,0.5,0.584440
4,64,30000,front,male,right,1,1,None,None,0.000000,0.0,0.610859
...,...,...,...,...,...,...,...,...,...,...,...,...
619,None,0,back,female,left,4,1,None,None,0.994964,0.0,0.994930
620,None,0,back,female,left,3,,1,None,0.998482,0.5,0.998439
621,None,0,back,female,left,2,1,None,None,0.985213,0.0,0.980931
622,None,0,back,female,left,1,1,None,None,0.965399,0.0,0.974257
